In [ ]:
PSNR AND THE SSIM FOR THE ORGINAL VIDEO AND THE OUTPUT VIDEO

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

def build_generator():
    """ Define a simple CNN generator for watermark prediction """
    inputs = tf.keras.layers.Input(shape=(128, 128, 1))
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = tf.keras.Model(inputs, x, name="Generator")
    return model

generator = build_generator()
weights_path = "./model_weights/generator_epoch_4900.weights.h5"

if os.path.exists(weights_path):
    generator.load_weights(weights_path)
    print("✅ Generator weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Weights file not found at {weights_path}")

INPUT_VIDEO_PATH = "./input1.mp4"
OUTPUT_VIDEO_PATH = "./output_watermarked1.mp4"


cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_count = 0
psnr_values = []
ssim_values = []
alpha = 0.01

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  
    frame = frame.astype(np.float32) / 255.0  
    gray_frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (128, 128)) / 255.0  
    input_frame = np.expand_dims(resized_frame, axis=[0, -1])
    watermark = generator.predict(input_frame)[0, :, :, 0]  
    watermark_resized = cv2.resize(watermark, (frame_width, frame_height))

    watermarked_frame = frame + alpha * np.expand_dims(watermark_resized, axis=-1)
    watermarked_frame = np.clip(watermarked_frame * 255, 0, 255).astype(np.uint8)

    out.write(watermarked_frame)

    original_frame = (frame * 255).astype(np.uint8)
    psnr_value = psnr(original_frame, watermarked_frame, data_range=255)
    ssim_value = ssim(original_frame, watermarked_frame, data_range=255, win_size=3, channel_axis=-1)
    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)
    frame_count += 1

cap.release()
out.release()

avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)


print(f"✅ Watermarked video saved: {OUTPUT_VIDEO_PATH}")
print(f"📊  PSNR: {avg_psnr:.4f} (More Than 30 is better)")
print(f"📊  SSIM : {avg_ssim:.6f} (Closer to 1 is better)")



✅ Generator weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━

In [ ]:
GAN GENERAED WATERMARK VS EXTRACTED WATERMARK SSIM 

In [2]:
import cv2
import numpy as np
import tensorflow as tf
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

def build_generator():
    """ Define a simple CNN generator for watermark prediction """
    inputs = tf.keras.layers.Input(shape=(128, 128, 1))
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    model = tf.keras.Model(inputs, x, name="Generator")
    return model

generator = build_generator()
weights_path = "./model_weights/generator_epoch_4900.weights.h5"

if os.path.exists(weights_path):
    generator.load_weights(weights_path)
    print("✅ Generator weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ Weights file not found at {weights_path}")

INPUT_VIDEO_PATH = "./input1.mp4"
OUTPUT_VIDEO_PATH = "./output_watermarked1.mp4"
GENERATED_WATERMARK_PATH = "./generated_watermarks"
EXTRACTED_WATERMARK_PATH = "./extracted_watermarks"

os.makedirs(GENERATED_WATERMARK_PATH, exist_ok=True)
os.makedirs(EXTRACTED_WATERMARK_PATH, exist_ok=True)

cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

frame_count = 0
psnr_values = []
ssim_values = []
watermark_ssim_values = []
watermark_mse_values = []

alpha = 0.07

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  

    frame = frame.astype(np.float32) / 255.0  

    gray_frame = cv2.cvtColor((frame * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (128, 128)) / 255.0  

    input_frame = np.expand_dims(resized_frame, axis=[0, -1])
    watermark = generator.predict(input_frame)[0, :, :, 0]  

    watermark_resized = cv2.resize(watermark, (frame_width, frame_height))

    generated_watermark_path = os.path.join(GENERATED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(generated_watermark_path, (watermark_resized * 255).astype(np.uint8))

    watermarked_frame = frame + alpha * np.expand_dims(watermark_resized, axis=-1)
    watermarked_frame = np.clip(watermarked_frame * 255, 0, 255).astype(np.uint8)

    out.write(watermarked_frame)

    original_frame = (frame * 255).astype(np.uint8)
    psnr_value = psnr(original_frame, watermarked_frame, data_range=255)
    ssim_value = ssim(original_frame, watermarked_frame, data_range=255, win_size=3, channel_axis=-1)

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)

    extracted_watermark = (watermarked_frame.astype(np.float32) - frame * 255) / alpha
    extracted_watermark = np.mean(extracted_watermark, axis=-1) 

    extracted_watermark = cv2.GaussianBlur(extracted_watermark, (3, 3), 0)

    extracted_watermark = np.clip(extracted_watermark, 0, 255).astype(np.uint8)
    
    extracted_watermark_path = os.path.join(EXTRACTED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(extracted_watermark_path, extracted_watermark)

    h, w = watermark_resized.shape
    h_start, h_end = h // 4, 3 * h // 4
    w_start, w_end = w // 4, 3 * w // 4
    cropped_original = watermark_resized[h_start:h_end, w_start:w_end]
    cropped_extracted = extracted_watermark[h_start:h_end, w_start:w_end]

    watermark_ssim = ssim(cropped_original, cropped_extracted.astype(np.float32) / 255, data_range=1.0)
    watermark_mse = mean_squared_error(cropped_original.flatten(), cropped_extracted.flatten() / 255)

    watermark_ssim_values.append(watermark_ssim)
    watermark_mse_values.append(watermark_mse)

    frame_count += 1

cap.release()
out.release()

avg_watermark_ssim = np.mean(watermark_ssim_values)
avg_watermark_mse = np.mean(watermark_mse_values)

print(f"✅ Watermarked video saved: {OUTPUT_VIDEO_PATH}")
print(f"✅ Generated watermarks saved in: {GENERATED_WATERMARK_PATH}")
print(f"✅ Extracted watermarks saved in: {EXTRACTED_WATERMARK_PATH}")
print(f"📊 Watermark SSIM: {avg_watermark_ssim:.4f} (Closer to 1 is better)")
print(f"📊 Watermark MSE: {avg_watermark_mse:.6f} (Lower is better)")
print(f"🎯 Model Accuracy (Based on SSIM): {avg_watermark_ssim * 100:.2f}%")


✅ Generator weights loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━

In [ ]:
EXTRACT THE WATERMARK FROM THE DEEPFAKED WATERMARK VIDEO

In [3]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim

ORIGINAL_VIDEO_PATH = "./input1.mp4"
WATERMARKED_VIDEO_PATH = "./deepfaked.mp4"
EXTRACTED_WATERMARK_PATH = "./deepfaked_extracted_watermarks"

os.makedirs(EXTRACTED_WATERMARK_PATH, exist_ok=True)

cap_original = cv2.VideoCapture(ORIGINAL_VIDEO_PATH)
cap_watermarked = cv2.VideoCapture(WATERMARKED_VIDEO_PATH)

alpha = 0.07 

frame_count = 0
ssim_values = []

while cap_original.isOpened() and cap_watermarked.isOpened():
    ret_orig, frame_orig = cap_original.read()
    ret_watermarked, frame_watermarked = cap_watermarked.read()

    if not ret_orig or not ret_watermarked:
        break 

    frame_orig = frame_orig.astype(np.float32)
    frame_watermarked = frame_watermarked.astype(np.float32)

    extracted_watermark = (frame_watermarked - frame_orig) / alpha

    extracted_watermark = np.mean(extracted_watermark, axis=-1)

    extracted_watermark = cv2.GaussianBlur(extracted_watermark, (3, 3), 0)

    extracted_watermark = np.clip(extracted_watermark, 0, 255).astype(np.uint8)

    extracted_watermark_path = os.path.join(EXTRACTED_WATERMARK_PATH, f"frame_{frame_count:04d}.png")
    cv2.imwrite(extracted_watermark_path, extracted_watermark)

    frame_count += 1

cap_original.release()
cap_watermarked.release()

print(f"✅ Extracted watermarks saved in: {EXTRACTED_WATERMARK_PATH}")


✅ Extracted watermarks saved in: ./deepfaked_extracted_watermarks


In [ ]:
Calculating the SSIM of the GAN Generated Watermark and the extracted watermark from deepfaked video

In [5]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error

GENERATED_WATERMARK_PATH = "./generated_watermarks"
EXTRACTED_WATERMARK_PATH = "./deepfaked_extracted_watermarks"

generated_files = sorted(os.listdir(GENERATED_WATERMARK_PATH))
extracted_files = sorted(os.listdir(EXTRACTED_WATERMARK_PATH))

assert len(generated_files) == len(extracted_files), "Mismatch in number of frames!"

SSIM_THRESHOLD = 0.6
MSE_THRESHOLD = 0.02 

deepfake_frames = 0
total_frames = len(generated_files)
ssim_values = []
mse_values = []

for gen_file, ext_file in zip(generated_files, extracted_files):
    gen_path = os.path.join(GENERATED_WATERMARK_PATH, gen_file)
    ext_path = os.path.join(EXTRACTED_WATERMARK_PATH, ext_file)

    gen_img = cv2.imread(gen_path, cv2.IMREAD_GRAYSCALE) / 255.0  
    ext_img = cv2.imread(ext_path, cv2.IMREAD_GRAYSCALE) / 255.0  

    ssim_value = ssim(gen_img, ext_img, data_range=1.0)
    mse_value = mean_squared_error(gen_img.flatten(), ext_img.flatten())

    ssim_values.append(ssim_value)
    mse_values.append(mse_value)

    if ssim_value < SSIM_THRESHOLD or mse_value > MSE_THRESHOLD:
        deepfake_frames += 1

avg_ssim = np.mean(ssim_values)
avg_mse = np.mean(mse_values)

deepfake_percentage = (deepfake_frames / total_frames) * 100


print(f"📊 Average SSIM: {avg_ssim:.4f} (Closer to 1 is better)")
print(f"📊 Average MSE: {avg_mse:.6f} (Lower is better)")
print(f"🔎 Deepfake Suspicious Frames: {deepfake_frames}/{total_frames} ({deepfake_percentage:.2f}%)")



📊 Average SSIM: 0.5828 (Closer to 1 is better)
📊 Average MSE: 0.027067 (Lower is better)
🔎 Deepfake Suspicious Frames: 469/469 (100.00%)


In [ ]:
Extracting the watermark stored in the dwt/dct 

In [6]:
import cv2
import numpy as np
import pywt
import os
from tqdm import tqdm

def extract_watermark_from_image(watermarked_img_path, alpha=0.05, watermark_shape=(50, 50)):
    wm_img = cv2.imread(watermarked_img_path, cv2.IMREAD_GRAYSCALE)
    if wm_img is None:
        raise ValueError(f"Could not load image: {watermarked_img_path}")
    coeffs = pywt.dwt2(wm_img, 'haar')
    LL, (LH, HL, HH) = coeffs
    dct_LL = cv2.dct(np.float32(LL))
    estimated_original = cv2.medianBlur(dct_LL, 5)
    watermark_diff = (dct_LL - estimated_original) / alpha
    watermark = cv2.resize(watermark_diff, watermark_shape)
    watermark = cv2.normalize(watermark, None, 0, 255, cv2.NORM_MINMAX)
    
    return watermark.astype(np.uint8)

def batch_extract_watermarks(input_dir, output_dir, alpha=0.05, watermark_shape=(50, 50)):
    """
    Processes all images in a directory to extract watermarks
    """
    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.startswith('frame_') and f.endswith('.png')])
    watermark_accumulator = None
    valid_count = 0
    for frame_file in tqdm(frame_files, desc="Extracting watermarks"):
        try:
            frame_path = os.path.join(input_dir, frame_file)
            wm = extract_watermark_from_image(frame_path, alpha, watermark_shape)
            if watermark_accumulator is None:
                watermark_accumulator = np.zeros_like(wm, dtype=np.float32)
            watermark_accumulator += wm.astype(np.float32)
            valid_count += 1
            output_path = os.path.join(output_dir, f"extracted_{frame_file}")
            cv2.imwrite(output_path, wm)
            
        except Exception as e:
            print(f"Error processing {frame_file}: {str(e)}")
    if valid_count > 0:
        avg_watermark = (watermark_accumulator / valid_count).astype(np.uint8)
        cv2.imwrite(os.path.join(output_dir, "averaged_watermark.png"), avg_watermark)
        print(f"Successfully processed {valid_count} frames. Average watermark saved.")
    else:
        print("No valid frames processed.")

if __name__ == "__main__":
    INPUT_DIR = "./generated_watermarks"  # Folder with your watermarked frames
    OUTPUT_DIR = "./check_generated"  # Where to save extracted watermarks
    ALPHA = 0.05  # Must match your embedding strength
    WATERMARK_SHAPE = (50, 50)  # Must match your original watermark dimensions
    
    # Run extraction
    batch_extract_watermarks(INPUT_DIR, OUTPUT_DIR, ALPHA, WATERMARK_SHAPE)

Extracting watermarks: 100%|█████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 75.00it/s]

Successfully processed 469 frames. Average watermark saved.


In [ ]:
Extracting the watermark embeded in the deepfaked frame

In [7]:
import cv2
import numpy as np
import pywt
import os
from tqdm import tqdm

def extract_watermark_from_image(watermarked_img_path, alpha=0.05, watermark_shape=(50, 50)):
    wm_img = cv2.imread(watermarked_img_path, cv2.IMREAD_GRAYSCALE)
    if wm_img is None:
        raise ValueError(f"Could not load image: {watermarked_img_path}")
    coeffs = pywt.dwt2(wm_img, 'haar')
    LL, (LH, HL, HH) = coeffs
    dct_LL = cv2.dct(np.float32(LL))
    estimated_original = cv2.medianBlur(dct_LL, 5)
    watermark_diff = (dct_LL - estimated_original) / alpha
    watermark = cv2.resize(watermark_diff, watermark_shape)
    watermark = cv2.normalize(watermark, None, 0, 255, cv2.NORM_MINMAX)
    
    return watermark.astype(np.uint8)

def batch_extract_watermarks(input_dir, output_dir, alpha=0.05, watermark_shape=(50, 50)):
    """
    Processes all images in a directory to extract watermarks
    """
    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.startswith('frame_') and f.endswith('.png')])
    watermark_accumulator = None
    valid_count = 0
    for frame_file in tqdm(frame_files, desc="Extracting watermarks"):
        try:
            frame_path = os.path.join(input_dir, frame_file)
            wm = extract_watermark_from_image(frame_path, alpha, watermark_shape)
            if watermark_accumulator is None:
                watermark_accumulator = np.zeros_like(wm, dtype=np.float32)
            watermark_accumulator += wm.astype(np.float32)
            valid_count += 1
            output_path = os.path.join(output_dir, f"extracted_{frame_file}")
            cv2.imwrite(output_path, wm)
            
        except Exception as e:
            print(f"Error processing {frame_file}: {str(e)}")
    if valid_count > 0:
        avg_watermark = (watermark_accumulator / valid_count).astype(np.uint8)
        cv2.imwrite(os.path.join(output_dir, "averaged_watermark.png"), avg_watermark)
        print(f"Successfully processed {valid_count} frames. Average watermark saved.")
    else:
        print("No valid frames processed.")

if __name__ == "__main__":
    INPUT_DIR = "./deepfaked_extracted_watermarks"  # Folder with your watermarked frames
    OUTPUT_DIR = "./deepfaked_extracted"  # Where to save extracted watermarks
    ALPHA = 0.05  # Must match your embedding strength
    WATERMARK_SHAPE = (50, 50)  # Must match your original watermark dimensions
    
    # Run extraction
    batch_extract_watermarks(INPUT_DIR, OUTPUT_DIR, ALPHA, WATERMARK_SHAPE)

Extracting watermarks: 100%|█████████████████████████████████████████████████████████| 469/469 [00:06<00:00, 76.90it/s]

Successfully processed 469 frames. Average watermark saved.


In [ ]:
Calculating the SSIM for the embeded (50*50)sized watermark before and after deepfake 

In [8]:
import cv2
import numpy as np
import os
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

def calculate_ssim(img1, img2):
    """Calculate SSIM between two images"""
    # Ensure images are the same size
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    # Convert to float32 for calculation
    img1 = img1.astype(np.float32)
    img2 = img2.astype(np.float32)
    
    # Calculate SSIM (multichannel=False for grayscale)
    return ssim(img1, img2, data_range=255, full=False)

def compare_folders(generated_dir, extracted_dir):
    """Compare SSIM between corresponding images in two folders"""
    # Get sorted list of files
    gen_files = sorted([f for f in os.listdir(generated_dir) if f.endswith('.png')])
    ext_files = sorted([f for f in os.listdir(extracted_dir) if f.endswith('.png')])
    
    # Verify we have matching files
    if len(gen_files) != len(ext_files):
        print(f"Warning: Folder count mismatch - Generated: {len(gen_files)}, Extracted: {len(ext_files)}")
    
    ssim_values = []
    
    for gen_file, ext_file in tqdm(zip(gen_files, ext_files), total=min(len(gen_files), len(ext_files))):
        gen_path = os.path.join(generated_dir, gen_file)
        ext_path = os.path.join(extracted_dir, ext_file)
        
        # Load images as grayscale
        gen_img = cv2.imread(gen_path, cv2.IMREAD_GRAYSCALE)
        ext_img = cv2.imread(ext_path, cv2.IMREAD_GRAYSCALE)
        
        if gen_img is None or ext_img is None:
            print(f"Skipping {gen_file} - couldn't load one or both images")
            continue
        
        # Calculate SSIM
        current_ssim = calculate_ssim(gen_img, ext_img)
        ssim_values.append(current_ssim)
        
        print(f"{gen_file} vs {ext_file}: SSIM = {current_ssim:.4f}")
    
    if ssim_values:
        avg_ssim = np.mean(ssim_values)
        print(f"\nAverage SSIM across {len(ssim_values)} frames: {avg_ssim:.4f}")
        return avg_ssim
    else:
        print("No valid image pairs found")
        return 0

if __name__ == "__main__":
    # Configuration
    GENERATED_DIR = "./check_generated"    # Folder with watermarked images
    EXTRACTED_DIR = "./deepfaked_extracted"    # Folder with extracted watermarks
    
    # Run comparison
    average_ssim = compare_folders(GENERATED_DIR, EXTRACTED_DIR)
    
    # Save results to file
    with open("ssim_results.txt", "w") as f:
        f.write(f"Average SSIM: {average_ssim:.4f}\n")

  6%|████▊                                                                           | 28/470 [00:00<00:01, 270.77it/s]

averaged_watermark.png vs averaged_watermark.png: SSIM = 0.9949
extracted_frame_0000.png vs extracted_frame_0000.png: SSIM = 0.9768
extracted_frame_0001.png vs extracted_frame_0001.png: SSIM = 0.9749
extracted_frame_0002.png vs extracted_frame_0002.png: SSIM = 0.9747
extracted_frame_0003.png vs extracted_frame_0003.png: SSIM = 0.9784
extracted_frame_0004.png vs extracted_frame_0004.png: SSIM = 0.9763
extracted_frame_0005.png vs extracted_frame_0005.png: SSIM = 0.9762
extracted_frame_0006.png vs extracted_frame_0006.png: SSIM = 0.9745
extracted_frame_0007.png vs extracted_frame_0007.png: SSIM = 0.9754
extracted_frame_0008.png vs extracted_frame_0008.png: SSIM = 0.9743
extracted_frame_0009.png vs extracted_frame_0009.png: SSIM = 0.9760
extracted_frame_0010.png vs extracted_frame_0010.png: SSIM = 0.9724
extracted_frame_0011.png vs extracted_frame_0011.png: SSIM = 0.9743
extracted_frame_0012.png vs extracted_frame_0012.png: SSIM = 0.9747
extracted_frame_0013.png vs extracted_frame_0013.png

 19%|███████████████▎                                                                | 90/470 [00:00<00:01, 278.61it/s]

extracted_frame_0050.png vs extracted_frame_0050.png: SSIM = 0.9727
extracted_frame_0051.png vs extracted_frame_0051.png: SSIM = 0.9695
extracted_frame_0052.png vs extracted_frame_0052.png: SSIM = 0.9736
extracted_frame_0053.png vs extracted_frame_0053.png: SSIM = 0.9738
extracted_frame_0054.png vs extracted_frame_0054.png: SSIM = 0.9713
extracted_frame_0055.png vs extracted_frame_0055.png: SSIM = 0.9710
extracted_frame_0056.png vs extracted_frame_0056.png: SSIM = 0.9702
extracted_frame_0057.png vs extracted_frame_0057.png: SSIM = 0.9700
extracted_frame_0058.png vs extracted_frame_0058.png: SSIM = 0.9700
extracted_frame_0059.png vs extracted_frame_0059.png: SSIM = 0.9693
extracted_frame_0060.png vs extracted_frame_0060.png: SSIM = 0.9678
extracted_frame_0061.png vs extracted_frame_0061.png: SSIM = 0.9687
extracted_frame_0062.png vs extracted_frame_0062.png: SSIM = 0.9660
extracted_frame_0063.png vs extracted_frame_0063.png: SSIM = 0.9670
extracted_frame_0064.png vs extracted_frame_0064

 33%|██████████████████████████▍                                                    | 157/470 [00:00<00:01, 309.36it/s]

...

extracted_frame_0335.png vs extracted_frame_0335.png: SSIM = 0.9687
extracted_frame_0336.png vs extracted_frame_0336.png: SSIM = 0.9721
extracted_frame_0337.png vs extracted_frame_0337.png: SSIM = 0.9749
extracted_frame_0338.png vs extracted_frame_0338.png: SSIM = 0.9735
extracted_frame_0339.png vs extracted_frame_0339.png: SSIM = 0.9761
extracted_frame_0340.png vs extracted_frame_0340.png: SSIM = 0.9749
extracted_frame_0341.png vs extracted_frame_0341.png: SSIM = 0.9774
extracted_frame_0342.png vs extracted_frame_0342.png: SSIM = 0.9724
extracted_frame_0343.png vs extracted_frame_0343.png: SSIM = 0.9755
extracted_frame_0344.png vs extracted_frame_0344.png: SSIM = 0.9749
extracted_frame_0345.png vs extracted_frame_0345.png: SSIM = 0.9740
extracted_frame_0346.png vs extracted_frame_0346.png: SSIM = 0.9741
extracted_frame_0347.png vs extracted_frame_0347.png: SSIM = 0.9757
extracted_frame_0348.png vs extracted_frame_0348.png: SSIM = 0.9692
extracted_frame_0349.png vs extracted_frame_0349

 94%|█████████████████████████████████████████████████████████████████████████▉     | 440/470 [00:01<00:00, 285.65it/s]

extracted_frame_0407.png vs extracted_frame_0407.png: SSIM = 0.9728
extracted_frame_0408.png vs extracted_frame_0408.png: SSIM = 0.9721
extracted_frame_0409.png vs extracted_frame_0409.png: SSIM = 0.9695
extracted_frame_0410.png vs extracted_frame_0410.png: SSIM = 0.9673
extracted_frame_0411.png vs extracted_frame_0411.png: SSIM = 0.9698
extracted_frame_0412.png vs extracted_frame_0412.png: SSIM = 0.9656
extracted_frame_0413.png vs extracted_frame_0413.png: SSIM = 0.9673
extracted_frame_0414.png vs extracted_frame_0414.png: SSIM = 0.9627
extracted_frame_0415.png vs extracted_frame_0415.png: SSIM = 0.9742
extracted_frame_0416.png vs extracted_frame_0416.png: SSIM = 0.9688
extracted_frame_0417.png vs extracted_frame_0417.png: SSIM = 0.9685
extracted_frame_0418.png vs extracted_frame_0418.png: SSIM = 0.9666
extracted_frame_0419.png vs extracted_frame_0419.png: SSIM = 0.9671
extracted_frame_0420.png vs extracted_frame_0420.png: SSIM = 0.9678
extracted_frame_0421.png vs extracted_frame_0421

100%|███████████████████████████████████████████████████████████████████████████████| 470/470 [00:01<00:00, 283.75it/s]

extracted_frame_0462.png vs extracted_frame_0462.png: SSIM = 0.9488
extracted_frame_0463.png vs extracted_frame_0463.png: SSIM = 0.9495
extracted_frame_0464.png vs extracted_frame_0464.png: SSIM = 0.9470
extracted_frame_0465.png vs extracted_frame_0465.png: SSIM = 0.9558
extracted_frame_0466.png vs extracted_frame_0466.png: SSIM = 0.9513
extracted_frame_0467.png vs extracted_frame_0467.png: SSIM = 0.9501
extracted_frame_0468.png vs extracted_frame_0468.png: SSIM = 0.9573

Average SSIM across 470 frames: 0.9629
